Unsupervised learning techniques are those that involve training with or analyzing unlabeled data. Dimensionality reduction, for instance, is an unsupervised learning task.

Other examples:

- **Clustering**: group similar instance together into clusters. Useful for data analysis, segmentation, recommender systems, serach engines, semi-supervised learning, and dimensionality reduction.

- **Anomaly Detection**: a predictor is trained to recognize 'normal' instances in order to detect abnormalities. Useful for defect detection and identifying new trends.

- **Density Estimation**: estimates the probability density function (PDF) of the random process that generated the dataset. Useful for anomaly detection, data analysis, and visualization.

Topics:

- Clustering
    - K-Means Algorithm
    - Segmentation
    - Preprocessing
    - Semi-Supervised Learning
    - DBSCAN Algorithm
    - Other Algorithms
- Gaussian Mixtures
    - Anomaly Detection
    - Bayesian Models
    - Other Algorithms

# Clustering

**Clustering** is the task of identifying similar instances and assigning them to clusters.

Some applications:

- **Customer segmentation**: clustering customers based on purchases and activity, and adapt products and marketing to each segment. Useful for recommender systems.
- **Data analysis**: clustering instances based on feature relationships. Useful for gaining early insights and separating the data.
- **Dimensionality reduction**: after clustering a dataset, each instance can be redefined according to its affinity with each cluster, reducing dimensionality to the (much lower) number of clusters while preserving most of the information.
- **Anomaly detection**: identifying instances with low cluster affinities to detect outliers.
- **Semi-supervised learning**: clustering a partially-labeled dataset, and then propagating the labels to like-instances for a subsequent supervised training stage.
- **Search engines**: clustering a collection of images, then returning the best-matching cluster when a reference image is provided.
- **Image segmentation**: clustering pixels according to color, then replacing each with its cluster's mean color. Useful for object and contour detection, tracking systems.